# Reactiva Perú 2022 — Portado a Jupyter (Python) guiado por tu código R

Este cuaderno **mantiene el formato de Jupyter** (mismas secciones) y **sustituye el código** por Python, siguiendo la intención del script R.


**Resultado del chequeo de finalidad:** equivalente ✅

### Qué se detectó en tu script R
- **Carga/Limpieza de datos**: detectado en R
- **EDA/Gráficos**: detectado en R
- **Tablas de contingencia**: detectado en R
- **Modelo Lasso/logístico**: detectado en R
- **Partición/validación**: detectado en R
- **Métricas**: detectadas en R

## 1) Carga de librerías y utilidades

In [ ]:
import os, glob, unicodedata, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

warnings.filterwarnings("ignore")

def normalize(s: str) -> str:
    s = unicodedata.normalize('NFD', s)
    return ''.join(c for c in s if unicodedata.category(c) != 'Mn').lower()


## 2) Importación de datos (equivalente al `read_excel` de R)

In [ ]:
# Busca el archivo Excel de Reactiva en /mnt/data/
excel_candidates = [p for p in glob.glob('/mnt/data/*.xlsx') if 'reactiva' in normalize(os.path.basename(p))]
if not excel_candidates:
    raise FileNotFoundError("No se encontró un archivo .xlsx con 'reactiva' en /mnt/data/.")
excel_path = excel_candidates[0]
print("Usando archivo:", excel_path)

df = pd.read_excel(excel_path, engine='openpyxl')
print("Dimensiones:", df.shape)
df.head(3)


## 3) Normalización de nombres de columnas y tipado

In [ ]:
def norm_cols(cols):
    out = []
    for c in cols:
        c0 = normalize(str(c)).replace('/', '_').replace('(', '').replace(')', '').replace('.', '_')
        c0 = '_'.join(c0.split())
        out.append(c0)
    return out

df.columns = norm_cols(df.columns)

aliases = {
    "ruc_o_dni": ["ruc_o_dni", "ruc_dni", "rucdni", "ruc__dni"],
    "razon_social": ["razon_social", "razon_social_"],
    "sector_economico": ["sector_economico"],
    "saldo_insoluto": ["saldo_insoluto_s", "saldo_insoluto", "saldo_insoluto_s_"],
    "cobertura_saldo_insoluto": ["cobertura_del_saldo_insoluto_s", "cobertura_saldo_insoluto_s", "cobertura_saldo_insoluto"],
    "entidad_otorgante_credito": ["nombre_de_entidad_otorgante_del_credito", "entidad_otorgante_credito"],
    "departamento": ["departamento"],
    "repro": ["repro"]
}
def pick_col(cands):
    for col in cands:
        if col in df.columns:
            return col
    return None
cols = {k: pick_col(v) for k,v in aliases.items()}
cols


## 4) EDA básico (gráficos/tablas)

In [ ]:
# Pie de REPRO
if cols["repro"] and df[cols["repro"]].notna().any():
    repro = df[cols["repro"]].astype(str).str.strip().str.upper().replace({'1':'SI','0':'NO'})
    vc = repro.value_counts(dropna=False)
    plt.figure()
    plt.pie(vc.values, labels=vc.index.astype(str), autopct='%1.1f%%')
    plt.title("Porcentaje de empresas con REPRO")
    plt.show()

# Barras por Departamento
if cols["departamento"]:
    s = df[cols["departamento"]].value_counts().sort_values(ascending=True)
    plt.figure()
    plt.barh(s.index.astype(str), s.values)
    plt.title("Frecuencia por Departamento")
    plt.xlabel("Frecuencia"); plt.ylabel("Departamento")
    plt.show()

# Tabla de contingencia Sector x REPRO
if cols["sector_economico"] and cols["repro"]:
    temp = df[[cols["sector_economico"], cols["repro"]]].copy()
    temp[cols["repro"]] = temp[cols["repro"]].astype(str).str.strip().str.upper().replace({'1':'SI','0':'NO'})
    ctab = pd.crosstab(temp[cols["sector_economico"]], temp[cols["repro"]])
    ctab.head()


## 5) Correlación (Spearman)

In [ ]:
if cols["saldo_insoluto"] and cols["cobertura_saldo_insoluto"]:
    corr_spear = df[[cols["saldo_insoluto"], cols["cobertura_saldo_insoluto"]]].corr(method='spearman').iloc[0,1]
    print(f"Spearman(SALDO_INSOLUTO, COBERTURA_SALDO_INSOLUTO) = {corr_spear:.4f}")
else:
    print("No se pudo calcular Spearman (faltan columnas numéricas).")


## 6) Modelo Lasso logístico con validación cruzada (equivalente a `cv.glmnet`)

In [ ]:
required_for_model = ["sector_economico", "entidad_otorgante_credito", "departamento", "saldo_insoluto", "repro"]
missing = [k for k in required_for_model if not cols[k]]
if missing:
    print("Modelo NO ejecutado: faltan columnas ->", missing)
else:
    y_raw = df[cols["repro"]]
    if y_raw.dtype.kind in "OUS":
        y = y_raw.astype(str).str.strip().str.upper().map({"SI":1, "NO":0, "1":1, "0":0})
    else:
        y = y_raw
    if y.isna().any():
        raise ValueError("REPRO contiene valores no mapeables a 0/1. Normaliza a {SI,NO} o {0,1}.")

    X = df[[cols["sector_economico"], cols["entidad_otorgante_credito"], cols["departamento"], cols["saldo_insoluto"]]].copy()
    cat_cols = [cols["sector_economico"], cols["entidad_otorgante_credito"], cols["departamento"]]
    num_cols = [cols["saldo_insoluto"]]

    pre = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols)
    ])

    clf = LogisticRegressionCV(
        Cs=10,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=123),
        penalty='l1',
        solver='saga',
        scoring='roc_auc',
        max_iter=5000,
        n_jobs=-1,
        refit=True
    )
    from sklearn.pipeline import Pipeline
    pipe = Pipeline([("pre", pre), ("clf", clf)])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=123)
    pipe.fit(X_train, y_train)

    y_prob = pipe.predict_proba(X_test)[:,1]
    y_pred = (y_prob > 0.5).astype(int)

    cm = confusion_matrix(y_test, y_pred)
    acc = (cm[0,0] + cm[1,1]) / cm.sum()
    auc = roc_auc_score(y_test, y_prob)

    print("Matriz de confusión:\n", cm)
    print(f"Accuracy: {acc:.4f} | ROC-AUC: {auc:.4f}")
    print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, digits=4))

    # Coeficientes no nulos (interpretación del lasso)
    ohe = pipe.named_steps["pre"].named_transformers_["cat"]
    feature_names = list(ohe.get_feature_names_out(cat_cols)) + num_cols
    coefs = pipe.named_steps["clf"].coef_.ravel()
    nz = [(n, w) for n, w in zip(feature_names, coefs) if abs(w) > 1e-8]
    for name, w in sorted(nz, key=lambda x: abs(x[1]), reverse=True)[:30]:
        print(f"{name:60s} {w: .4f}")


## 7) Notas de compatibilidad y qué haría falta si algo no coincide

- **Finalidad**: tu flujo en R y este en Python son **equivalentes** (carga → EDA → tablas → correlación → Lasso CV).  
- Si en tu R hay **transformaciones específicas** (por ejemplo, crear `REPRO` a partir de otra columna, imputaciones o filtros), agrégalas **antes** del bloque de modelado.  
- Si tu R usa **métricas distintas** o **otro umbral** (p.ej. 0.4 en vez de 0.5), ajusta la línea `y_pred = (y_prob > 0.5)`.  
- Si tu R balancea clases (SMOTE/weights), en Python podrías usar `class_weight='balanced'` en `LogisticRegressionCV`.
